In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
import os
import numpy as np
from shapely.geometry import box
from tqdm.notebook import tqdm

# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

# 경고 보이기  
#warnings.filterwarnings(action='default')

# 월별 테마반경
thema_buffer_SKT_Y( ) : SKT 유동인구 있을 때 사용<br/>
thema_buffer_SKT_N( ) : SKT 유동인구 없을 때 사용

In [6]:
def thema_buffer_SKT_Y(file_name, year, encoding='cp949'):
    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")
    
    
    wifi_thema = ['거북공원', '김해민속박물관', '김해박물관', '김해시민의종', '대성동고분박물관',
                  '먹자골목', '박물관역', '봉황대공원', '봉황역', '수로왕릉', '수로왕릉역', 
                  '수로왕비릉', '신세계백화점', '연지공원']
    
    # 병합할 포인트
    
    User_point = pd.read_csv(f'./전처리/grid/{file_name}.csv', encoding=encoding)
    
    User_point['x'] = None
    User_point['y'] = None

    for n in tqdm(range(len(User_point))):
        xy = User_point['geom'][n].split()
        User_point['x'][n] = float(xy[1].split('(')[1])
        User_point['y'][n] = float(xy[2].split(')')[0])
    
    User_point.drop('geom', axis=1, inplace=True)
    User_point['geometry'] = User_point.apply(lambda row : Point([row['x'], row['y']]), axis=1)
    
    User_point = gpd.GeoDataFrame(User_point, geometry='geometry', crs = epsg5179)
    
    # 테마 디렉토리 리스트
    
    thema_path = './원본데이터/구역_반경/'
    thema_list = os.listdir(thema_path)
    
    for tm in tqdm(thema_list):
        # 테마 구역 리스트
        thema_in =  os.listdir(thema_path + tm)
        
        # 병합을 위한 변수
        cnct_df = None
        
        for tm_i in thema_in:
            
            # 김해시 테마 반경
            thema_polygon = gpd.GeoDataFrame.from_file(thema_path + tm + f'/{tm_i}/{tm_i}.shp', 
                                                     encoding=encoding)
            thema_polygon.crs = epsg5179
            
            # 공간 병합
            result = gpd.sjoin(thema_polygon, User_point, how='left', op="intersects")

            result.drop(['index_right', 'gid', 'recordid', 'hcode','x','y'], axis=1, inplace=True)

            result.reset_index(drop=True, inplace=True)
            
            result['sum_pop'] = int(result['sum_pop'].sum())
            result['w_pop'] = int(result['w_pop'].sum())
            result['v_pop'] = int(result['v_pop'].sum())
            result['h_pop'] = int(result['h_pop'].sum())
            
            result.drop([n for n in range(1, len(result))], axis=0, inplace=True)
            
            
            month = file_name.split('_')[-1]
            
            month_0 = int(month)
            
#            if month_0 < 10:
#                month_0 = f'0{month_0}'
            
            if tm_i in wifi_thema:
                wifi_df = pd.read_csv(f'../김해시_Wi-Fi/전처리/250m_반경_구역/{year}년/{month_0}월/{tm_i}.csv', encoding='cp949')
                result =  pd.concat([result, wifi_df],axis=1)
            else:
                result['Wi-Fi_user'] = 0
            
            
            # 폴더 생성
            try:
                directory = f'D:/김해시청 프로젝트/김해시_유동인구_구역/월별_반경_데이터_shp/{year}년/{month}월/{tm}/{tm_i}/'
                if not os.path.exists(directory):
                    os.makedirs(directory)
            except OSError:
                print ('Error: Creating directory. ' +  directory)
        
            
            result = result.astype({'distance':int, 'year':int, 'month':int})
            
            
            result = result[['MEAN_X', 'MEAN_Y', 'ringId', 'distance', 'geometry', 'year',
                             'month','h_pop', 'w_pop', 'v_pop', 'sum_pop','Wi-Fi_user']]
            
            result['layer'] = tm_i
            
            # shp파일 저장
            result.to_file(f'{directory}{tm_i}.shp',encoding='cp949', driver='ESRI Shapefile')
            
            
            # 데이터프레임 병합
            if tm_i == thema_in[0]:
                cnct_df = result
                continue
            cnct_df = pd.concat([cnct_df, result])
            
        # 폴더 생성
        try:
            directory2 = f'D:/김해시청 프로젝트/김해시_유동인구_구역/월별_반경_데이터_병합_shp/{year}년/{month}월/{tm}/'
            if not os.path.exists(directory2):
                os.makedirs(directory2)
        except OSError:
            print ('Error: Creating directory. ' +  directory2)
            
        cnct_df.to_file(f'{directory2}{tm}.shp', encoding='cp949', driver='ESRI Shapefile')

In [7]:
# 월별 반경
for n in tqdm(range(3, 11)):
    thema_buffer_SKT_Y(f'agg_pcell_time_service_pop_2022_{n}', 2022)

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/74465 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/75963 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/77181 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/77381 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
def thema_buffer_SKT_N(year, month, encoding='cp949'):
    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")
    
    
    wifi_thema = ['거북공원', '김해민속박물관', '김해박물관', '김해시민의종', '대성동고분박물관',
                  '먹자골목', '박물관역', '봉황대공원', '봉황역', '수로왕릉', '수로왕릉역', 
                  '수로왕비릉', '신세계백화점', '연지공원']
    
    # 테마 디렉토리 리스트
    
    thema_path = './원본데이터/구역_반경/'
    thema_list = os.listdir(thema_path)
    
    for tm in tqdm(thema_list):
        # 테마 구역 리스트
        thema_in =  os.listdir(thema_path + tm)
        
        # 병합을 위한 변수
        cnct_df = None
        
        for tm_i in thema_in:
            
            # 김해시 테마 반경
            thema_polygon = gpd.GeoDataFrame.from_file(thema_path + tm + f'/{tm_i}/{tm_i}.shp', 
                                                     encoding=encoding)
            thema_polygon.crs = epsg5179
            

            

            month_0 = month
                 
            if tm_i in wifi_thema:
                wifi_df = pd.read_csv(f'../김해시_Wi-Fi/전처리/250m_반경_구역/{year}년/{month_0}월/{tm_i}.csv', encoding='cp949')
                thema_polygon =  pd.concat([thema_polygon, wifi_df],axis=1)
            else:
                thema_polygon['Wi-Fi_user'] = 0
            
            
            # 폴더 생성
            try:
                directory = f'D:/김해시청 프로젝트/김해시_유동인구_구역/월별_반경_데이터_shp/{year}년/{month}월/{tm}/{tm_i}/'
                if not os.path.exists(directory):
                    os.makedirs(directory)
            except OSError:
                print ('Error: Creating directory. ' +  directory)
        
            
            thema_polygon = thema_polygon.astype({'distance':int})
            
            thema_polygon['layer'] = tm_i
            
            # shp파일 저장
            thema_polygon.to_file(f'{directory}{tm_i}.shp',encoding='cp949', driver='ESRI Shapefile')
            
            
            # 데이터프레임 병합
            if tm_i == thema_in[0]:
                cnct_df = thema_polygon
                continue
            cnct_df = pd.concat([cnct_df, thema_polygon])
            
        # 폴더 생성
        try:
            directory2 = f'D:/김해시청 프로젝트/김해시_유동인구_구역/월별_반경_데이터_병합_shp/{year}년/{month}월/{tm}/'
            if not os.path.exists(directory2):
                os.makedirs(directory2)
        except OSError:
            print ('Error: Creating directory. ' +  directory2)
            
        cnct_df.to_file(f'{directory2}{tm}.shp', encoding='cp949', driver='ESRI Shapefile')

In [9]:
for n in tqdm(range(1, 13)):
    thema_buffer_SKT_N(2021, n)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

# 연도별 테마 반경
year_thema_buffer_SKT_Y( ) : SKT 유동인구 있을 때 사용 <br/>
year_thema_buffer_SKT_N( ) : SKT 유동인구 없을 때 사용

In [10]:
def year_thema_buffer_SKT_Y(file_name, year, encoding='cp949'):
    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")
    
    
    wifi_thema = ['거북공원', '김해민속박물관', '김해박물관', '김해시민의종', '대성동고분박물관',
                  '먹자골목', '박물관역', '봉황대공원', '봉황역', '수로왕릉', '수로왕릉역', 
                  '수로왕비릉', '신세계백화점', '연지공원']
    
    # 병합할 포인트
    
    User_point = pd.read_csv(f'./전처리/grid/{file_name}.csv', encoding=encoding)
    
    User_point['x'] = None
    User_point['y'] = None

    for n in tqdm(range(len(User_point))):
        xy = User_point['geom'][n].split()
        User_point['x'][n] = float(xy[1].split('(')[1])
        User_point['y'][n] = float(xy[2].split(')')[0])
    
    User_point.drop('geom', axis=1, inplace=True)
    User_point['geometry'] = User_point.apply(lambda row : Point([row['x'], row['y']]), axis=1)
    
    User_point = gpd.GeoDataFrame(User_point, geometry='geometry', crs = epsg5179)
    
    # 테마 디렉토리 리스트
    
    thema_path = './원본데이터/구역_반경/'
    thema_list = os.listdir(thema_path)
    
    for tm in tqdm(thema_list):
        # 테마 구역 리스트
        thema_in =  os.listdir(thema_path + tm)
        
        # 병합을 위한 변수
        cnct_df = None
        
        for tm_i in thema_in:
            
            # 김해시 테마 반경
            thema_polygon = gpd.GeoDataFrame.from_file(thema_path + tm + f'/{tm_i}/{tm_i}.shp', 
                                                     encoding=encoding)
            thema_polygon.crs = epsg5179
            
            # 공간 병합
            result = gpd.sjoin(thema_polygon, User_point, how='left', op="intersects")

            result.drop(['index_right', 'gid', 'recordid','x','y'], axis=1, inplace=True)

            result.reset_index(drop=True, inplace=True)
            
            result['sum_pop'] = int(result['sum_pop'].sum())
            result['w_pop'] = int(result['w_pop'].sum())
            result['v_pop'] = int(result['v_pop'].sum())
            result['h_pop'] = int(result['h_pop'].sum())
            
            result.drop([n for n in range(1, len(result))], axis=0, inplace=True)
            
            
            if tm_i in wifi_thema:
                wifi_df = pd.read_csv(f'../김해시_Wi-Fi/전처리/250m_반경_구역/{year}년/{year}년/{tm_i}.csv', encoding='cp949')
                result =  pd.concat([result, wifi_df],axis=1)
            else:
                result['Wi-Fi_user'] = 0
            
            
            # 폴더 생성
            try:
                directory = f'D:/김해시청 프로젝트/김해시_유동인구_구역/월별_반경_데이터_shp/{year}년/{year}년/{tm}/{tm_i}/'
                if not os.path.exists(directory):
                    os.makedirs(directory)
            except OSError:
                print ('Error: Creating directory. ' +  directory)
        
            
            result = result.astype({'distance':int})
            
            
            result = result[['MEAN_X', 'MEAN_Y', 'ringId', 'distance', 'geometry','h_pop', 'w_pop', 'v_pop', 'sum_pop','Wi-Fi_user']]
            
            result['layer'] = tm_i
            
            # shp파일 저장
            result.to_file(f'{directory}{tm_i}.shp',encoding='cp949', driver='ESRI Shapefile')
            
            
            # 데이터프레임 병합
            if tm_i == thema_in[0]:
                cnct_df = result
                continue
            cnct_df = pd.concat([cnct_df, result])
            
        # 폴더 생성
        try:
            directory2 = f'D:/김해시청 프로젝트/김해시_유동인구_구역/월별_반경_데이터_병합_shp/{year}년/{year}년/{tm}/'
            if not os.path.exists(directory2):
                os.makedirs(directory2)
        except OSError:
            print ('Error: Creating directory. ' +  directory2)
            
        cnct_df.to_file(f'{directory2}{tm}.shp', encoding='cp949', driver='ESRI Shapefile')

In [11]:
year_thema_buffer_SKT_Y('agg_pcell_time_service_pop_2022', 2022)

  0%|          | 0/79935 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [45]:
def year_thema_buffer_SKT_N(year, encoding='cp949'):
    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")
    
    
    wifi_thema = ['거북공원', '김해민속박물관', '김해박물관', '김해시민의종', '대성동고분박물관',
                  '먹자골목', '박물관역', '봉황대공원', '봉황역', '수로왕릉', '수로왕릉역', 
                  '수로왕비릉', '신세계백화점', '연지공원']
    
    
    # 테마 디렉토리 리스트
    
    thema_path = './원본데이터/구역_반경/'
    thema_list = os.listdir(thema_path)
    
    for tm in tqdm(thema_list):
        # 테마 구역 리스트
        thema_in =  os.listdir(thema_path + tm)
        
        # 병합을 위한 변수
        cnct_df = None
        
        for tm_i in thema_in:
            
            # 김해시 테마 반경
            thema_polygon = gpd.GeoDataFrame.from_file(thema_path + tm + f'/{tm_i}/{tm_i}.shp', 
                                                     encoding=encoding)
            thema_polygon.crs = epsg5179
            
            
            if tm_i in wifi_thema:
                wifi_df = pd.read_csv(f'../김해시_Wi-Fi/전처리/250m_반경_구역/{year}년/{year}년/{tm_i}.csv', encoding='cp949')
                thema_polygon =  pd.concat([thema_polygon, wifi_df],axis=1)
            else:
                thema_polygon['Wi-Fi_user'] = 0
            thema_polygon['layer'] = tm_i
            
            
            # 폴더 생성
            try:
                directory = f'D:/김해시청 프로젝트/김해시_유동인구_구역/월별_반경_데이터_shp/{year}년/{year}년/{tm}/{tm_i}/'
                if not os.path.exists(directory):
                    os.makedirs(directory)
            except OSError:
                print ('Error: Creating directory. ' +  directory)
        
            
            thema_polygon = thema_polygon.astype({'distance':int})
            
            
            # shp파일 저장
            thema_polygon.to_file(f'{directory}{tm_i}.shp',encoding='cp949', driver='ESRI Shapefile')
            
            
            # 데이터프레임 병합
            if tm_i == thema_in[0]:
                cnct_df = thema_polygon
                continue
            cnct_df = pd.concat([cnct_df, thema_polygon])
            
        # 폴더 생성
        try:
            directory2 = f'D:/김해시청 프로젝트/김해시_유동인구_구역/월별_반경_데이터_병합_shp/{year}년/{year}년/{tm}/'
            if not os.path.exists(directory2):
                os.makedirs(directory2)
        except OSError:
            print ('Error: Creating directory. ' +  directory2)
            
        cnct_df.to_file(f'{directory2}{tm}.shp', encoding='cp949', driver='ESRI Shapefile')

In [46]:
year_thema_buffer_SKT_N(2021)

  0%|          | 0/5 [00:00<?, ?it/s]

In [45]:
csv_list = os.listdir('D:/김해시청 프로젝트/김해시_유동인구_구역/shp_csv')


concat_df = None

for n in range(len(csv_list)):
    df = pd.read_csv('D:/김해시청 프로젝트/김해시_유동인구_구역/shp_csv/' + csv_list[n], encoding='cp949')
    
    if n == 0:
        concat_df = df
        continue
    concat_df = pd.concat([concat_df, df])


In [49]:
concat_df.drop(['MEAN_X', 'MEAN_Y', 'ringId', 'distance'], axis=1, inplace=True)

In [51]:
concat_df.to_csv('D:/김해시청 프로젝트/김해시_유동인구_구역/shp_csv/2022_3~10월_유동인구.csv', encoding='cp949', index=False)

# 성연령별

In [3]:
def year_thema_buffer_sex_age(file_name, year, encoding='cp949'):
    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    epsg5179 = from_string("+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=GRS80 +units=m +no_defs")
    
    
    wifi_thema = ['거북공원', '김해민속박물관', '김해박물관', '김해시민의종', '대성동고분박물관',
                  '먹자골목', '박물관역', '봉황대공원', '봉황역', '수로왕릉', '수로왕릉역', 
                  '수로왕비릉', '신세계백화점', '연지공원']
    
    # 병합할 포인트
    
    User_point = pd.read_csv(f'./전처리/grid/{file_name}.csv', encoding=encoding)
    
    User_point['x'] = None
    User_point['y'] = None

    for n in tqdm(range(len(User_point))):
        xy = User_point['geom'][n].split()
        User_point['x'][n] = float(xy[1].split('(')[1])
        User_point['y'][n] = float(xy[2].split(')')[0])
    
    User_point.drop('geom', axis=1, inplace=True)
    User_point['geometry'] = User_point.apply(lambda row : Point([row['x'], row['y']]), axis=1)
    
    User_point = gpd.GeoDataFrame(User_point, geometry='geometry', crs = epsg5179)
    
    # 테마 디렉토리 리스트
    
    thema_path = './원본데이터/구역_반경/'
    thema_list = os.listdir(thema_path)
    
    for tm in tqdm(thema_list):
        # 테마 구역 리스트
        thema_in =  os.listdir(thema_path + tm)
        
        # 병합을 위한 변수
        cnct_df = None
        
        for tm_i in thema_in:
            
            # 김해시 테마 반경
            thema_polygon = gpd.GeoDataFrame.from_file(thema_path + tm + f'/{tm_i}/{tm_i}.shp', 
                                                     encoding=encoding)
            thema_polygon.crs = epsg5179
            
            # 공간 병합
            result = gpd.sjoin(thema_polygon, User_point, how='left', op="intersects")

            result.drop(['index_right', 'gid', 'recordid','x','y'], axis=1, inplace=True)

            result.reset_index(drop=True, inplace=True)
            
            result['m_0009'] = int(result['m_0009'].sum())
            result['m_1019'] = int(result['m_1019'].sum())
            result['m_2029'] = int(result['m_2029'].sum())
            result['m_3039'] = int(result['m_3039'].sum())
            result['m_4049'] = int(result['m_4049'].sum())
            result['m_5059'] = int(result['m_5059'].sum())
            result['m_6069'] = int(result['m_6069'].sum())
            result['m_7000'] = int(result['m_7000'].sum())
            result['m_sum'] = int(result['m_sum'].sum())
            result['w_0009'] = int(result['w_0009'].sum())
            result['w_1019'] = int(result['w_1019'].sum())
            result['w_2029'] = int(result['w_2029'].sum())
            result['w_3039'] = int(result['w_3039'].sum())
            result['w_4049'] = int(result['w_4049'].sum())
            result['w_5059'] = int(result['w_5059'].sum())
            result['w_6069'] = int(result['w_6069'].sum())            
            result['w_7000'] = int(result['w_7000'].sum())
            result['w_sum'] = int(result['w_sum'].sum())
            result['t_sum'] = int(result['t_sum'].sum())                 
            
            result.drop([n for n in range(1, len(result))], axis=0, inplace=True)
            
            
            
            # 폴더 생성
            try:
                directory = f'D:/김해시청 프로젝트/김해시_유동인구_구역/월별_반경_데이터_shp/{year}년/성연령별/{tm}/{tm_i}/'
                if not os.path.exists(directory):
                    os.makedirs(directory)
            except OSError:
                print ('Error: Creating directory. ' +  directory)
        
            
            result = result.astype({'distance':int})
            
            
            result['layer'] = tm_i
            
            # shp파일 저장
            result.to_file(f'{directory}{tm_i}.shp',encoding='cp949', driver='ESRI Shapefile')
            
            
            # 데이터프레임 병합
            if tm_i == thema_in[0]:
                cnct_df = result
                continue
            cnct_df = pd.concat([cnct_df, result])
            
        # 폴더 생성
        try:
            directory2 = f'D:/김해시청 프로젝트/김해시_유동인구_구역/월별_반경_데이터_병합_shp/{year}년/성연령별/{tm}/'
            if not os.path.exists(directory2):
                os.makedirs(directory2)
        except OSError:
            print ('Error: Creating directory. ' +  directory2)
            
        cnct_df.to_file(f'{directory2}{tm}.shp', encoding='cp949', driver='ESRI Shapefile')

In [4]:
year_thema_buffer_sex_age('agg_pcell_month_sex_age_pop_2022', 2022)

  0%|          | 0/80299 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [19]:
path = 'D:\김해시청 프로젝트\김해시_유동인구_구역\월별_반경_데이터_병합_shp/2021년/'
path = path.split('\\')
path = '/'.join(path)
path

'D:/김해시청 프로젝트/김해시_유동인구_구역/월별_반경_데이터_병합_shp/2021년/'

In [18]:
#--------------------------------------
# 2021년
#--------------------------------------

cnct_df_1 = None
cnct_df_2 = None

print('전체 진행도')
for n in tqdm(range(1,13)):
    path = 'D:\김해시청 프로젝트\김해시_유동인구_구역\월별_반경_데이터_병합_shp/2021년/'
    path = path.split('\\')
    path = '/'.join(path)
    
    path = path + f'{n}월/'
    
    thema_list = os.listdir(path)
    
    print(f'{n}월 진행도')
    for m in tqdm(range(len(thema_list))):
        tm = thema_list[m]
        thema_polygon = gpd.GeoDataFrame.from_file(path + f'{tm}/{tm}.shp', 
                                                     encoding='cp949')
        thema_polygon['thema'] = tm
        thema_polygon['year'] = 2021
        thema_polygon['month'] = n
        
        if m == 0:
            cnct_df_1 = thema_polygon
            continue
        cnct_df_1 = pd.concat([cnct_df_1, thema_polygon])
    
    if n == 1:
        cnct_df_2 = cnct_df_1
        continue
    cnct_df_2 = pd.concat([cnct_df_2, cnct_df_1])
    
cnct_df_2['h_pop'] = 0
cnct_df_2['w_pop'] = 0
cnct_df_2['v_pop'] = 0
cnct_df_2['sum_pop'] = 0

cnct_df_2 = cnct_df_2[list(cnct_df_4.columns)]            

전체 진행도


  0%|          | 0/12 [00:00<?, ?it/s]

1월 진행도


  0%|          | 0/5 [00:00<?, ?it/s]

2월 진행도


...

  0%|          | 0/5 [00:00<?, ?it/s]

11월 진행도


  0%|          | 0/5 [00:00<?, ?it/s]

12월 진행도


  0%|          | 0/5 [00:00<?, ?it/s]

In [17]:
#--------------------------------------
# 2022년
#--------------------------------------

cnct_df_3 = None
cnct_df_4 = None

print('전체 진행도')
for n in tqdm(range(3,11)):
    path = 'D:\김해시청 프로젝트\김해시_유동인구_구역\월별_반경_데이터_병합_shp/2022년/'
    path = path.split('\\')
    path = '/'.join(path)
    
    path = path + f'{n}월/'
    
    thema_list = os.listdir(path)
    
    print(f'{n}월 진행도')
    for m in tqdm(range(len(thema_list))):
        tm = thema_list[m]
        thema_polygon = gpd.GeoDataFrame.from_file(path + f'{tm}/{tm}.shp', 
                                                     encoding='cp949')
        thema_polygon['thema'] = tm
        thema_polygon.drop(['year', 'month'], axis=1, inplace=True)
        thema_polygon['year'] = 2022
        thema_polygon['month'] = n
        
        
        
        if m == 0:
            cnct_df_3 = thema_polygon
            continue
        cnct_df_3 = pd.concat([cnct_df_3, thema_polygon])
    
    if n == 1:
        cnct_df_4 = cnct_df_1
        continue
    cnct_df_4 = pd.concat([cnct_df_4, cnct_df_3])
    

전체 진행도


  0%|          | 0/8 [00:00<?, ?it/s]

3월 진행도


  0%|          | 0/5 [00:00<?, ?it/s]

4월 진행도


...

  0%|          | 0/5 [00:00<?, ?it/s]

9월 진행도


  0%|          | 0/5 [00:00<?, ?it/s]

10월 진행도


  0%|          | 0/5 [00:00<?, ?it/s]

In [19]:
cnct_df_5 = pd.concat([cnct_df_2, cnct_df_4])

In [20]:
cnct_df_5.reset_index(drop=True, inplace=True)

In [21]:
cnct_df_5.head()

MEAN_X     MEAN_Y  ringId  distance  h_pop  w_pop  v_pop  sum_pop  \
0  128.871806  35.240167       1       250      0      0      0        0   
1    128.8743  35.227239       1       250      0      0      0        0   
2  128.872111  35.232889       1       250      0      0      0        0   
3  128.865611  35.236497       1       250      0      0      0        0   
4  128.874028  35.231565       1       250      0      0      0        0   

   Wi-Fi_user  layer                                           geometry thema  \
0      104599   박물관역  POLYGON ((1125066.730 1694724.446, 1125066.537...   경전철   
1      370771    봉황역  POLYGON ((1125313.482 1693293.687, 1125313.289...   경전철   
2       73058  수로왕릉역  POLYGON ((1125105.638 1693917.618, 1125105.445...   경전철   
3      296806   거북공원  POLYGON ((1124508.627 1694309.594, 1124508.435...    공원   
4      400677  봉황대공원  POLYGON ((1125420.074 1693780.088, 1125419.881...    공원   

   year  month  
0  2021      1  
1  2021      1  
2  2021      1  
3  2021      1  
4  2021      1

In [68]:
cnct_df_5.columns

Index(['MEAN_X', 'MEAN_Y', 'ringId', 'distance', 'h_pop', 'w_pop', 'v_pop',
       'sum_pop', 'Wi-Fi_user', 'layer', 'geometry', 'thema', 'year', 'month'],
      dtype='object')

In [71]:
df_group = pd.DataFrame(cnct_df_5.groupby(['thema', 'year', 'month'])['Wi-Fi_user'].sum())
df_group

Wi-Fi_user
thema year month            
경전철   2021 1          765085
           2          702618
           3          463031
           4          451555
           5          411686
...                      ...
특성화거리 2022 6          503445
           7          504865
           8          484131
           9          483767
           10         511976

[100 rows x 1 columns]

In [96]:
pdf2 = pd.pivot_table(df_group,                # 피벗할 데이터프레임
                     index = ['year','month'],    # 행 위치에 들어갈 열
                     columns = 'thema',    # 열 위치에 들어갈 열
                     values = 'Wi-Fi_user')     # 데이터로 사용할 열
pdf2

thema          경전철       공원  대형복합쇼핑몰   역사 관광지    특성화거리
year month                                            
2021 1      765085  1841538   496145  1271223  1569163
     2      702618  1739501   434024  1248464  1389002
     3      463031  1201796   266226   946706   982946
     4      451555  1065388   246896   884336   885736
     5      411686  1002837   238752   868325   847519
     6      320239   753820   175365   659294   621614
     7      296395   740532   169490   635694   592873
     8      186848   458541   112730   409239   371304
     9      238905   548422   128846   499840   444188
     10     234985   555578   130043   506629   444031
     11     209366   376367    29910   443942   388398
     12     347975   660702   101502   728490   657793
2022 3      295772   690629   174796   677585   550384
     4      124010   275682    67731   256861   207425
     5      336906   773180   187488   699767   544383
     6      317452   756644   172873   681023   503445
     7      307525   754741   178323   699693   504865
     8      298486   737349   180768   718262   484131
     9      289491   730362   183518   711813   483767
     10     313894   747439   195868   786786   511976

In [97]:
pdf2.to_excel('../김해시_Wi-Fi/전처리/피벗_테마_월별_와이파이_이용자.xlsx', encoding='cp949')

In [76]:
df_group.to_excel('../김해시_Wi-Fi/전처리/테마_월별_와이파이_이용자.xlsx', encoding='cp949')

In [81]:
df_layer_group = pd.DataFrame(cnct_df_5.groupby(['layer', 'year', 'month'])['Wi-Fi_user'].sum())
df_layer_group.reset_index(inplace=True)

In [82]:
df_layer_group.columns

Index(['layer', 'year', 'month', 'Wi-Fi_user'], dtype='object')

In [94]:
t_li = list(cnct_df_5['layer'][:18])

pdf1 = pd.pivot_table(df_layer_group,                # 피벗할 데이터프레임
                     index = ['year','month'],    # 행 위치에 들어갈 열
                     columns = 'layer',    # 열 위치에 들어갈 열
                     values = 'Wi-Fi_user')     # 데이터로 사용할 열
pdf1 = pdf1[t_li]
pdf1

layer         박물관역     봉황역  수로왕릉역    거북공원   봉황대공원    연지공원  문화의전당 인근  신세계백화점  \
year month                                                                    
2021 1      138139  541031  85915  510518  603586  727434         0  496145   
     2      130309  479959  92350  478097  550708  710696         0  434024   
     3       84330  330547  48154  324473  344151  533172         0  266226   
     4       89092  309375  53088  284309  324903  456176         0  246896   
     5       88837  278647  44202  264508  303521  434808         0  238752   
     6       64988  218579  36672  191582  226938  335300         0  175365   
     7       59178  207416  29801  192695  224560  323277         0  169490   
     8       38764  128510  19574  122612  140134  195795         0  112730   
     9       51867  157372  29666  149440  167556  231426         0  128846   
     10      51522  155860  27603  147688  172124  235766         0  130043   
     11      41260  147013  21093   71834  161595  142938         0   29910   
     12      71291  243688  32996  114368  297229  249105         0  101502   
2022 3       67701  199686  28385  157347  230956  302326         0  174796   
     4       30288   84703   9019   63913   91650  120119         0   67731   
     5       76464  232821  27621  178093  258827  336260         0  187488   
     6       72347  219950  25155  173434  232441  350769         0  172873   
     7       71126  208316  28083  173860  233816  347065         0  178323   
     8       64533  208476  25477  162159  235816  339374         0  180768   
     9       61092  199212  29187  166602  232295  331465         0  183518   
     10      63590  223782  26522  172697  252295  322447         0  195868   

layer       아이스퀘어  김해민속박물관   김해박물관  김해시민의종  대성동고분박물관    수로왕릉   수로왕비릉  \
year month                                                             
2021 1          0   135272  200764  274562    347889  177715  135021   
     2          0   180661  191696  248651    330835  166334  130287   
     3          0   138150  137175  161561    256848  131888  121084   
     4          0   134753  115803  147768    246668  130084  109260   
     5          0   133275  116879  147199    247383  120267  103322   
     6          0   111976   85904  106162    191695   89853   73704   
     7          0   109824   81950  105044    186821   80426   71629   
     8          0    73909   48215   55247    130009   54635   47224   
     9          0    84824   66374   70407    154198   64391   59646   
     10         0    83903   62685   86487    142702   71813   59039   
     11         0    72216   55113   93225    101043   71456   50889   
     12         0   109211  106785  134113    182741  121522   74118   
2022 3          0   111089   81360  116806    198676  100322   69332   
     4          0    42194   29440   30359     89474   35275   30119   
     5          0   122805   94726   68990    245312   87298   80636   
     6          0   113352   76745   99129    234935   76142   80720   
     7          0   104596   79739  116088    232251   83477   83542   
     8          0   110586   89950  104625    244521   88203   80377   
     9          0   115818   76724  105447    240244   95811   77769   
     10         0   112479   98714  120813    256455  117610   80715   

layer       글로벌 푸드타운     먹자골목  봉리단길  
year month                           
2021 1             0  1569163     0  
     2             0  1389002     0  
     3             0   982946     0  
     4             0   885736     0  
     5             0   847519     0  
     6             0   621614     0  
     7             0   592873     0  
     8             0   371304     0  
     9             0   444188     0  
     10            0   444031     0  
     11            0   388398     0  
     12            0   657793     0  
2022 3             0   550384     0  
     4             0   207425     0  
     5             0   544383     0  
     6             0   503445 

In [95]:
pdf1.to_excel('../김해시_Wi-Fi/전처리/구역_월별_와이파이_이용자.xlsx', encoding='cp949')

In [29]:
#--------------------------------------
# wi-fi 범례 설정
#--------------------------------------
tm = ['경전철', '공원', '대형복합쇼핑몰', '역사 관광지', '특성화거리']
cnct_df_5_0 = cnct_df_5[cnct_df_5['Wi-Fi_user']>0]

for n in tm:
    mn = cnct_df_5_0[cnct_df_5_0['thema'] == n]['Wi-Fi_user'].min()
    mx = cnct_df_5_0[cnct_df_5_0['thema'] == n]['Wi-Fi_user'].max()
    mean = (mn + mx) // 2
    
    print(n + f'(min) : {mn}')
    print(n + f'(max) : {mx}')
    print(n + f'(mean) : {mean}')
    print('-----------------------------------------')

경전철(min) : 5955
경전철(max) : 370771
경전철(mean) : 188363
-----------------------------------------
공원(min) : 15315
공원(max) : 431455
공원(mean) : 223385
-----------------------------------------
대형복합쇼핑몰(min) : 17805
대형복합쇼핑몰(max) : 319151
대형복합쇼핑몰(mean) : 168478
-----------------------------------------
역사 관광지(min) : 3887
역사 관광지(max) : 195028
역사 관광지(mean) : 99457
-----------------------------------------
특성화거리(min) : 52695
특성화거리(max) : 938144
특성화거리(mean) : 495419
-----------------------------------------


In [41]:
cnct_df_5_group = cnct_df_5[['thema','layer','year','Wi-Fi_user']]

In [42]:
df_group_sum = pd.DataFrame(cnct_df_5_group.groupby(['year','thema','layer'])['Wi-Fi_user'].sum())

In [43]:
df_group_sum.reset_index(inplace=True)

In [44]:
#--------------------------------------
# wi-fi 범례 설정
#--------------------------------------
tm = ['경전철', '공원', '대형복합쇼핑몰', '역사 관광지', '특성화거리']

for n in tm:
    mn = df_group_sum[df_group_sum['thema'] == n]['Wi-Fi_user'].min()
    mx = df_group_sum[df_group_sum['thema'] == n]['Wi-Fi_user'].max()
    mean = (mn + mx) // 2
    
    print(n + f'(min) : {mn}')
    print(n + f'(max) : {mx}')
    print(n + f'(mean) : {mean}')
    print('-----------------------------------------')

경전철(min) : 118416
경전철(max) : 1566151
경전철(mean) : 842283
-----------------------------------------
공원(min) : 217106
공원(max) : 1591705
공원(mean) : 904405
-----------------------------------------
대형복합쇼핑몰(min) : 0
대형복합쇼핑몰(max) : 1042959
대형복합쇼핑몰(mean) : 521479
-----------------------------------------
역사 관광지(min) : 54426
역사 관광지(max) : 815048
역사 관광지(mean) : 434737
-----------------------------------------
특성화거리(min) : 0
특성화거리(max) : 3163291
특성화거리(mean) : 1581645
-----------------------------------------


In [102]:
cnct_df_2022 = cnct_df_5[cnct_df_5['year']==2022]

In [105]:
df_group = pd.DataFrame(cnct_df_2022.groupby(['thema', 'year', 'month'])['sum_pop'].sum())
df_group

sum_pop
thema   year month          
경전철     2022 3       7046021
             4       6942131
             5       7141200
             6       6831412
             7       7080969
             8       7072773
             9       6779414
             10      6851956
공원      2022 3       6526005
             4       6935926
             5       7113047
             6       6893154
             7       7218479
             8       7306562
             9       7066671
             10      8295724
대형복합쇼핑몰 2022 3      12559740
             4      12196781
             5      12859942
             6      12398056
             7      13678969
             8      13648640
             9      12518578
             10     12384841
역사 관광지  2022 3       5344117
             4       5021577
             5       5210491
             6       4994392
             7       4186887
             8       4127588
             9       4122565
             10      4461172
특성화거리   2022 3      13427030
             4      12840282
             5      14494234
             6      14098044
             7      12700730
             8      12839806
             9      12497877
             10     13124527

In [107]:
pdf2 = pd.pivot_table(df_group,                # 피벗할 데이터프레임
                     index = ['year','month'],    # 행 위치에 들어갈 열
                     columns = 'thema',    # 열 위치에 들어갈 열
                     values = 'sum_pop')     # 데이터로 사용할 열
pdf2

thema           경전철       공원   대형복합쇼핑몰   역사 관광지     특성화거리
year month                                               
2022 3      7046021  6526005  12559740  5344117  13427030
     4      6942131  6935926  12196781  5021577  12840282
     5      7141200  7113047  12859942  5210491  14494234
     6      6831412  6893154  12398056  4994392  14098044
     7      7080969  7218479  13678969  4186887  12700730
     8      7072773  7306562  13648640  4127588  12839806
     9      6779414  7066671  12518578  4122565  12497877
     10     6851956  8295724  12384841  4461172  13124527

In [108]:
pdf2.to_excel('../김해시_Wi-Fi/전처리/피벗_테마_월별_와이파이_이용자.xlsx', encoding='cp949')

In [110]:
df_layer_group = pd.DataFrame(cnct_df_2022.groupby(['layer', 'year', 'month'])['sum_pop'].sum())
df_layer_group.reset_index(inplace=True)

In [111]:
t_li = list(cnct_df_5['layer'][:18])

pdf1 = pd.pivot_table(df_layer_group,                # 피벗할 데이터프레임
                     index = ['year','month'],    # 행 위치에 들어갈 열
                     columns = 'layer',    # 열 위치에 들어갈 열
                     values = 'sum_pop')     # 데이터로 사용할 열
pdf1 = pdf1[t_li]
pdf1

layer          박물관역      봉황역    수로왕릉역     거북공원   봉황대공원     연지공원  문화의전당 인근  \
year month                                                                  
2022 3      2278522  1514306  3253193  4522460  464521  1539024   3074824   
     4      2343798  1603711  2994622  4722186  439250  1774490   3405049   
     5      2414152  1678847  3048201  4887597  462565  1762885   3516906   
     6      2285344  1601091  2944977  4772175  448257  1672722   3379020   
     7      2158635  1667920  3254414  4731553  442378  2044548   3806401   
     8      2125989  1700152  3246632  4909318  437846  1959398   3688993   
     9      2039731  1677066  3062617  4838610  415522  1812539   3431299   
     10     2166009  1738494  2947453  5935738  423806  1936180   3420530   

layer        신세계백화점    아이스퀘어  김해민속박물관    김해박물관   김해시민의종  대성동고분박물관    수로왕릉  \
year month                                                                  
2022 3      5716725  3768191   330169  1153519  1661446   1130197  467417   
     4      5121259  3670473   310498  1073613  1579799   1077404  435344   
     5      5337461  4005575   320477  1115641  1627479   1107575  447364   
     6      5123095  3895941   314372  1058535  1556282   1063564  435827   
     7      5922340  3950228   288689   889636  1252155    872769  447152   
     8      5928047  4031600   287166   864925  1237888    859302  452409   
     9      5389717  3697562   273734   865468  1248567    853237  431993   
     10     5188544  3775767   289425   917428  1402534    947805  464450   

layer        수로왕비릉  글로벌 푸드타운     먹자골목    봉리단길  
year month                                     
2022 3      601369   4327325  8275569  824136  
     4      544919   4048038  7973518  818726  
     5      591955   4226673  9439122  828439  
     6      565812   4095952  9204219  797873  
     7      436486   4366089  7566734  767907  
     8      425898   4332124  7736987  770695  
     9      449566   4058854  7672088  766935  
     10     439530   4379244  7964791  780492

In [112]:
pdf1.to_excel('../김해시_Wi-Fi/전처리/구역_월별_와이파이_이용자.xlsx', encoding='cp949')

In [103]:
#--------------------------------------
# 통신사 범례 설정
#--------------------------------------
tm = ['경전철', '공원', '대형복합쇼핑몰', '역사 관광지', '특성화거리']

for n in tm:
    mn = cnct_df_2022[cnct_df_2022['thema'] == n]['sum_pop'].min()
    mx = cnct_df_2022[cnct_df_2022['thema'] == n]['sum_pop'].max()
    mean = (mn + mx) // 2
    
    print(n + f'(min) : {mn}')
    print(n + f'(max) : {mx}')
    print(n + f'(mean) : {mean}')
    print('-----------------------------------------')

경전철(min) : 1514306
경전철(max) : 3254414
경전철(mean) : 2384360
-----------------------------------------
공원(min) : 415522
공원(max) : 5935738
공원(mean) : 3175630
-----------------------------------------
대형복합쇼핑몰(min) : 3074824
대형복합쇼핑몰(max) : 5928047
대형복합쇼핑몰(mean) : 4501435
-----------------------------------------
역사 관광지(min) : 273734
역사 관광지(max) : 1661446
역사 관광지(mean) : 967590
-----------------------------------------
특성화거리(min) : 766935
특성화거리(max) : 9439122
특성화거리(mean) : 5103028
-----------------------------------------
